In [41]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [45]:
# mech:
R_w,G,M,K_w,C_rr,Rho,S_f,C_x,K_sign = sym.symbols('R_w,G,M,K_w,C_rr,Rho,S_f,C_x,K_sign',real=True)
tau_r,beta = sym.symbols('tau_r,beta',real=True)
nu,x_pos = sym.symbols('nu,x_pos',real=True)
snu,omega_r,tau_r_ref = sym.symbols('snu,omega_r,tau_r_ref',real=True)


# motor + dc/dc converter:
R_a,L_a,R_f,L_f,L_af,N_pp,K_p_a,K_i_a = sym.symbols('R_a,L_a,R_f,L_f,L_af,N_pp,K_p_a,K_i_a',real=True)
i_a,i_f,xi_a = sym.symbols('i_a,i_f,xi_a',real=True)
v_a,v_f,eta_a = sym.symbols('v_a,v_f,eta_a',real=True)

# battery:
e_b,soc,v_b1,v_b2,i_b,v_b,soc_ref,i_ch,xi_b = sym.symbols('e_b,soc,v_b1,v_b2,i_b,v_b,soc_ref,i_ch,xi_b',real=True)
C_b,R_b,K_p_b,K_i_b = sym.symbols('C_b,R_b,K_p_b,K_i_b',real=True)
E_b0,E_b1 = sym.symbols('E_b0,E_b1',real=True)
v_a = eta_a*v_b
i_b = eta_a*i_a

tau_r = L_af*i_a*i_f
tau_w = tau_r/K_w
f_w = tau_w/R_w
f_b = G*M*sym.sin(beta)
f_d = 0.5*Rho*S_f*C_x*nu**2*snu
f_r = C_rr*G*M*snu
p_r = tau_r*omega_r

dnu = 1/M*(f_w - f_d - f_r - f_b)

eq_snu = -snu + (1/(1+sym.exp(-nu*K_sign)))*2-1
eq_omega_r =-omega_r +  nu/(K_w*R_w)

omega_e = N_pp*omega_r
di_a = 1/L_a*(v_a - R_a*i_a - omega_e*L_af*i_f)
di_f = 1/L_f*(v_f - R_f*i_f)

epsilon_a = tau_r_ref - tau_r
dxi_a = epsilon_a
eq_eta_a = -eta_a + K_p_a * epsilon_a + K_i_a*xi_a

e_b = (E_b1 - E_b0)*soc + E_b0
epsilon_b = soc_ref - soc
dsoc = -(i_b-i_ch)/C_b
dxi_b = epsilon_b
eq_v_b = -v_b + e_b - R_b*i_b
eq_i_ch  = -i_ch + K_p_b * epsilon_b + K_i_b*xi_b


u_ini_dict = {'tau_r_ref':0.0,'beta':0.0,'v_f':220.0,'soc_ref':0.5}  # for the initialization problem
u_run_dict = {'tau_r_ref':0.0,'beta':0.0,'v_f':220.0,'soc_ref':0.5}  # for the running problem (here initialization and running problem are the same)


params_dict = {'R_w':0.316,'G':9.81,'M':1200.0,'K_w':1.0,'C_rr':0.03,'Rho':1.225,'S_f':2.13,'C_x':0.32,'K_sign':100,
               'R_a':0.06727,'L_a':0.001882,'R_f':30.72,'L_f':3.166,'L_af':0.2641,'N_pp':1.0, 'K_p_a':1.0,'K_i_a':1.0,
              'C_b':1e5,'R_b':0.1,'K_p_b':1e-5,'K_i_b':1e-5,'E_b0':300,'E_b1':320}


In [46]:
x_list = [ nu, i_a, i_f, xi_a, soc, xi_b]
f_list = [dnu,di_a,di_f,dxi_a,dsoc,dxi_b]
y_list = [   snu,   omega_r,   eta_a,   i_ch,   v_b]
g_list = [eq_snu,eq_omega_r,eq_eta_a,eq_i_ch,eq_v_b]
h_dict = {'f_w':f_w,'f_d':f_d,'f_r':f_r,'tau_r':tau_r,'p_r':p_r,'nu_kmh':3.6*nu,'v_b':v_b,'i_b':i_b}

sys_dict = {'name':f"ev1",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

bldr = db.builder(sys_dict);
bldr.build()

In [1]:
print(bldr.source)

NameError: name 'bldr' is not defined